In [1]:
import random

# ==== RATING CALCULATION ====

def compute_rating(wins, losses, runs_scored, runs_allowed):
    games = wins + losses
    # Handle case where games is 0 to prevent ZeroDivisionError
    if games == 0:
        return 0.0
    win_pct = wins / games
    run_diff_per_game = (runs_scored - runs_allowed) / games
    run_diff_norm = (run_diff_per_game + 3) / 6  # Normalize to [0, 1]
    # Use ** for exponentiation, not ^
    pythag = (runs_scored ** 2) / ((runs_scored ** 2) + (runs_allowed ** 2))
    return (0.25 * win_pct) + (0.25 * run_diff_norm) + (5 * pythag)


# ==== GAME & SERIES SIMULATION ====

def game_win_prob(team_rating, opp_rating, home_field_advantage=0):
    # Handle case where both ratings are 0 to prevent ZeroDivisionError
    if (team_rating + home_field_advantage + opp_rating) == 0:
        return 0.5 # Default to 50/50 if no rating information
    return (team_rating + home_field_advantage) / (team_rating + home_field_advantage + opp_rating)

def simulate_game_100_times(team_rating, opp_rating, is_home, hfa=0.05):
    team_wins = 0
    opp_wins = 0
    for _ in range(100): # Simulate each game 100 times
        prob = game_win_prob(team_rating, opp_rating, hfa if is_home else 0)
        if random.random() < prob:
            team_wins += 1
        else:
            opp_wins += 1
    return team_wins > opp_wins # Return True if team wins more simulations, False otherwise

def get_home_games(round_name, home_team_is_A):
    formats = {
        "wildcard":       [True, True, True],
        "divisional":     [True, True, False, False, True],
        "lcs":            [True, True, False, False, False, True, True],
        "world_series":   [True, True, False, False, False, True, True],
    }
    base = formats[round_name]
    return base if home_team_is_A else [not g for g in base]

def simulate_series(team_A, team_B, round_name):
    series_format = {
        "wildcard":     (2, 3),
        "divisional":   (3, 5),
        "lcs":          (4, 7),
        "world_series": (4, 7)
    }[round_name]

    wins_needed, max_games = series_format

    # HOME FIELD LOGIC
    if round_name == "world_series":
        # For World Series, the home field advantage is determined by the team with the better regular season record.
        # If records are tied, then by seed.
        if team_A["wins"] != team_B["wins"]:
            home_team_is_A = team_A["wins"] > team_B["wins"]
        else:
            home_team_is_A = team_A["seed"] < team_B["seed"]
    else:
        home_team_is_A = team_A["seed"] < team_B["seed"]

    home_games = get_home_games(round_name, home_team_is_A)
    home_team_in_series = team_A if home_team_is_A else team_B
    away_team_in_series = team_B if home_team_is_A else team_A


    team_A_wins = 0
    team_B_wins = 0

    print(f"\n🎯 {round_name.upper()} SERIES: {team_A['name']} (Seed {team_A['seed']}) vs {team_B['name']} (Seed {team_B['seed']})")
    print(f"🏟 Home field advantage: {home_team_in_series['name']}")

    for i in range(max_games):
        # Ensure 'i' is within the bounds of 'home_games' list
        if i >= len(home_games):
            break

        is_home_for_A = home_games[i]
        
        # Determine which team is home for this specific game
        current_home_team = team_A if is_home_for_A else team_B

        winner_A = simulate_game_100_times(team_A['rating'], team_B['rating'], is_home_for_A)

        if winner_A:
            team_A_wins += 1
            print(f"Game {i+1} (Home: {current_home_team['name']}): {team_A['name']} win.")
        else:
            team_B_wins += 1
            print(f"Game {i+1} (Home: {current_home_team['name']}): {team_B['name']} win.")

        if team_A_wins == wins_needed:
            print(f"🏆 {team_A['name']} win the {round_name} series {team_A_wins}-{team_B_wins}")
            return team_A
        elif team_B_wins == wins_needed:
            print(f"🏆 {team_B['name']} win the {round_name} series {team_B_wins}-{team_A_wins}")
            return team_B


# ==== POSTSEASON BRACKET SIMULATION ====

def simulate_postseason(teams_AL, teams_NL):
    print("\n⚾ SIMULATING MLB POSTSEASON BRACKET\n")

    # Wild Card Round
    ALWC1 = simulate_series(teams_AL[3], teams_AL[6], "wildcard")
    ALWC2 = simulate_series(teams_AL[4], teams_AL[5], "wildcard")
    NLWC1 = simulate_series(teams_NL[3], teams_NL[6], "wildcard")
    NLWC2 = simulate_series(teams_NL[4], teams_NL[5], "wildcard")

    # Division Series
    ALDS1 = simulate_series(teams_AL[1], ALWC1, "divisional")
    ALDS2 = simulate_series(teams_AL[2], ALWC2, "divisional")
    NLDS1 = simulate_series(teams_NL[1], NLWC1, "divisional")
    NLDS2 = simulate_series(teams_NL[2], NLWC2, "divisional")

    # League Championship
    ALCS = simulate_series(ALDS1, ALDS2, "lcs")
    NLCS = simulate_series(NLDS1, NLDS2, "lcs")

    # World Series
    WS_winner = simulate_series(ALCS, NLCS, "world_series")
    print(f"\n🏁 WORLD SERIES CHAMPION: {WS_winner['name']} 🏆\n")


# ==== TEAM DATA ====

def create_team(name, wins, losses, runs_scored, runs_allowed, seed):
    return {
        "name": name,
        "wins": wins,
        "losses": losses,
        "runs_scored": runs_scored,
        "runs_allowed": runs_allowed,
        "seed": seed,
        "rating": compute_rating(wins, losses, runs_scored, runs_allowed)
    }

teams_AL = [
    None, # 0 index is unused, keeping for 1-based seed indexing
    create_team("Toronto Blue Jays", 63, 43, 501, 459, 1),
    create_team("Detroit Tigers", 61, 46, 508, 446, 2),
    create_team("Houston Astros", 60, 46, 456, 417, 3),
    create_team("New York Yankees", 57, 48, 543, 448, 4),
    create_team("Boston Red Sox", 57, 50, 520, 466, 5),
    create_team("Seattle Mariners", 56, 50, 484, 464, 6),
]

teams_NL = [
    None, # 0 index is unused, keeping for 1-based seed indexing
    create_team("Chicago Cubs", 62, 43, 553, 437, 1),
    create_team("New York Mets", 62, 44, 466, 411, 2),
    create_team("Los Angeles Dodgers", 61, 45, 556, 490, 3),
    create_team("Milwaukee Brewers", 62, 43, 497, 405, 4),
    create_team("Philadelphia Phillies", 60, 45, 493, 419, 5),
    create_team("San Diego Padres", 57, 49, 426, 406, 6),
]

simulate_postseason(teams_AL, teams_NL)


⚾ SIMULATING MLB POSTSEASON BRACKET


🎯 WILDCARD SERIES: Houston Astros (Seed 3) vs Seattle Mariners (Seed 6)
🏟 Home field advantage: Houston Astros
Game 1 (Home: Houston Astros): Seattle Mariners win.
Game 2 (Home: Houston Astros): Houston Astros win.
Game 3 (Home: Houston Astros): Houston Astros win.
🏆 Houston Astros win the wildcard series 2-1

🎯 WILDCARD SERIES: New York Yankees (Seed 4) vs Boston Red Sox (Seed 5)
🏟 Home field advantage: New York Yankees
Game 1 (Home: New York Yankees): Boston Red Sox win.
Game 2 (Home: New York Yankees): New York Yankees win.
Game 3 (Home: New York Yankees): New York Yankees win.
🏆 New York Yankees win the wildcard series 2-1

🎯 WILDCARD SERIES: Los Angeles Dodgers (Seed 3) vs San Diego Padres (Seed 6)
🏟 Home field advantage: Los Angeles Dodgers
Game 1 (Home: Los Angeles Dodgers): Los Angeles Dodgers win.
Game 2 (Home: Los Angeles Dodgers): San Diego Padres win.
Game 3 (Home: Los Angeles Dodgers): Los Angeles Dodgers win.
🏆 Los Angeles Dodgers wi